<a href="https://colab.research.google.com/github/hanhluukim/replication-topic-modelling-in-embedding-space/blob/main/etm_in_julia/ETM_in_julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vorbereiten den Julia Enwicklungsfeld auf Colab**

In [ ]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

In [ ]:
VERSION

v"1.6.1"

#Optional GPU Experiments

In [ ]:
using Pkg
Pkg.add(["BenchmarkTools", "CUDA"])


using BenchmarkTools, CUDA

if has_cuda_gpu()
  print("The GPU device is:", CUDA.device())
end

The GPU device is:CuDevice(0)

In [ ]:
mcpu = rand(2^10, 2^10)
@benchmark mcpu*mcpu

BenchmarkTools.Trial: 79 samples with 1 evaluation.
 Range (min … max):  60.195 ms … 131.440 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     61.417 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   63.776 ms ±   8.926 ms  ┊ GC (mean ± σ):  0.71% ± 1.59%

   ▆█▄                                                          
  ▄███▆▃▃▃▃▅▃▃▃▃▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▁▃ ▁
  60.2 ms         Histogram: frequency by time         83.5 ms <

 Memory estimate: 8.00 MiB, allocs estimate: 2.

In [ ]:
println("The CuArrray operation should take around 0.5 ms(excluding CUDA downloading time which is a one time process), and should be much faster. If so, the GPU is working.")
mgpu = cu(mcpu)
@benchmark CUDA.@sync mgpu*mgpu

The CuArrray operation should take around 0.5 ms(excluding CUDA downloading time which is a one time process), and should be much faster. If so, the GPU is working.


BenchmarkTools.Trial: 2826 samples with 1 evaluation.
 Range (min … max):  1.190 ms …   1.172 s  ┊ GC (min … max): 0.00% … 0.48%
 Time  (median):     1.376 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.825 ms ± 22.021 ms  ┊ GC (mean ± σ):  0.11% ± 0.01%

      ▃██▃                                                    
  ▃▃▃▃████▅▃▂▂▂▂▂▂▂▁▂▁▂▂▁▂▁▂▁▁▁▂▂▁▁▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▂▁▂▁▁▂▁▂ ▃
  1.19 ms        Histogram: frequency by time        2.97 ms <

 Memory estimate: 256 bytes, allocs estimate: 9.

# **Importieren von gebrauchten Paketten für die Implementierung**

In [ ]:
using Pkg
Pkg.add(["BenchmarkTools", "CUDA", "Flux", "TextAnalysis"])

In [ ]:
Pkg.add("MAT")

In [ ]:
Pkg.add("DataLoaders")

In [ ]:
Pkg.add("Torch")

In [ ]:
using BenchmarkTools, CUDA
using Flux, TextAnalysis
using MAT
using DataLoaders
using Torch
using Torch: torch

┌ Info: Precompiling Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1317
┌ Info: Precompiling Torch [6a2ea274-3061-11ea-0d63-ff850051a295]
└ @ Base loading.jl:1317


# **Lesen Vokabular: id2word**

In [4]:
id2word = matread("/content/id2word.mat")
println(id2word["id"])
println(id2word["word"])
dict_id2word = Dict()
for i in id2word["id"]
  dict_id2word[i] = id2word["word"][i+1]
end

[0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 2

# **Lesen vorverarbeitete BoW-Repräsentationen aus der Datei: prepared_data/bow_train.mat**


1.   Zuerst werden die vorverarbeiteten Dateien auf diesem Colab hochladen
2.   Listeneintrag



In [68]:
# covert to bows-representation
function covert_to_bows(data, vocab_size, normalize_data)
  # covert
  n_docs = length(data["tokens"]) #list of [list of token-ids] = n_docs
  # matrix in julia V*D
  bows = zeros(Float64, vocab_size, n_docs)
  #print(size(bows))
  for docid in 1:n_docs
    doc_tokens = data["tokens"][docid]
    doc_counts = data["counts"][docid]
    for (index, token_id) in enumerate(doc_tokens)
      #replace zeros with count
      bows[token_id+1, docid] = doc_counts[index]
    end
  end

  doc_bow_sums = sum(bows, dims =1)
  # normalize
  if normalize_data
    #println("\n normalized")
    bows = bows ./ doc_bow_sums
  end
  #println(size(bows))
  return bows
end
# loading files
bow_train = matread("/content/bow_train.mat")["train"]
#bow_train["train"]["tokens"]
#bow_train["train"]["counts"]

batch_size = 50
vocab_size = length(id2word["id"])
print("vocab-size: " * string(vocab_size) * "\n") 

normalize_data = true
normalized_data = covert_to_bows(bow_train, vocab_size, normalize_data)
dataloader = DataLoader(normalized_data, batch_size)
print("check n-batches: " * string(length(dataloader)) * "\n")
for bow_doc_batch in dataloader
    println(size(bow_doc_batch))
    #@assert size(bow_doc_batch) == (vocab_size, batch_size)
end

vocab-size: 348
check n-batches: 3
(348, 50)
(348, 50)
(348, 39)


# **Lesen Word-Embedding word2vec aus der Datei: prepared_data/vocab_embedding.txt**

In [ ]:
embeddings = []
open("/content/vocab_embedding.txt") do file
  data = readlines(file)
  for line in data
    word, vector = split(line, "\t")
    vector = split(vector, " ")
    _vector = []
    for e in vector
      push!(_vector, parse(Float64, e))
    end
    push!(embeddings, _vector)
  end
end
#println(embeddings)
println("number of words in vocabulary: " * string(length(embeddings)))
print("dimension of word-embedding: " * string(length(embeddings[1])))

number of words in vocabulary: 348
dimension of word-embedding: 300

# **Implementieren von ETM Modell**

# **Evaluieren von Topics mittels folgenden Evaluationmaßen**